In [ ]:
# !df -h
# !cat /proc/cpuinfo
# !cat /proc/meminfo

In [1]:
!pip install -U sentence-transformers rank_bm25

     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 2.1MB 6.6MB/s 
     |████████████████████████████████| 1.2MB 18.8MB/s 
     |████████████████████████████████| 870kB 31.5MB/s 
     |████████████████████████████████| 3.3MB 35.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=f2d182a0de18d24886ef59f38f8acc67dec8a3e6a6c6b5eceabf6ad00d5ebfb9
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=5bb816e32324739c2ffad2308b6e3a0f939126c1eb36d333cad7ee867c056c94
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sentence-transformers sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch

if not torch.cuda.is_available():
  print("Warning: No GPU found. Please add GPU to your notebook")


#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
model_name = 'msmarco-distilbert-base-v2'

# model_name = "/content/drive/MyDrive/SBRT_output/training_biker_distilroberta_base_bi-encoder-min_50distilroberta-base-2021-02-23_02-33-56"
# model_name = "/content/drive/MyDrive/SBRT_output/training_biker_bi-encoder-min_5_max_10_ir_10_distilroberta-base_3_iter"
# model_name = "/content/drive/MyDrive/SBRT_output/training_biker_bi-encoder-min_5_max_10_ir_10_distilroberta-base-full-best"
bi_encoder = SentenceTransformer(model_name)
top_k = 100     #Number of passages we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('/content/drive/MyDrive/SBRT_output/training_biker_cross-encoder-30_iter_TinyBERT-full-best')


In [15]:
### Now we read the MS Marco dataset
data_folder = '/content/drive/MyDrive/biker_data/min_5_max_10_ir_10_30k'

os.makedirs(data_folder, exist_ok=True)
import json 

# in the order of 1 to 30k
corpus = []
collection_filepath = os.path.join(data_folder, 'Corpus_dict.json')
#"evaluate_Corpus_min_2_max_10.json"
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  data = json.load(fIn)
  for k in range(len(data)):
    corpus.append(data[str(k)])

# in the order of 1 to 30k
Answers = []
collection_filepath = os.path.join(data_folder, 'Answers_dict.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  data = json.load(fIn)
  for k in range(len(data)):
    Answers.append(data[str(k)])


evaluate_corpus = []
evaluate_answers = []
collection_filepath = os.path.join(data_folder, 'evaluate_Corpus.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  data = json.load(fIn)
  for k in data:
    evaluate_corpus.append(corpus[k])
    evaluate_answers.append(Answers[k])

queries = []
queries_answers = []
collection_filepath = os.path.join(data_folder, 'evaluate_queries.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  data = json.load(fIn)
  for k in data:
    queries.append(corpus[k])
    queries_answers.append(Answers[k])


In [16]:
import pandas as pd

In [26]:
df_test= pd.read_csv("/content/drive/MyDrive/biker_data/min_5_max_10_ir_10_30k/Biker_test_filtered.csv")
# # df_test= pd.read_csv("/content/drive/MyDrive/biker_data/min_5_max_10_ir_10_30k/BIKER_querys_final.csv")


queries = df_test["title"].to_list()
queries_answers = df_test["answer"].to_list()
queries_answers=[str(list(eval(x))) for x in queries_answers]

In [27]:
len(queries)

252

In [28]:
queries[:10]

['How to convert binary string value to decimal',
 'How can I use html styles in strings with format arguments',
 'Converting a sentence string to a string array of words in Java',
 'How do I create a parent-last / child-first ClassLoader in Java, or How to override an old Xerces version that was already loaded in the parent CL',
 'How to modify a Collection while iterating using for-each loop without ConcurrentModificationException',
 'How to generate a random permutation in Java',
 'Java generating Strings with placeholders',
 'Collect Linux command output',
 'How to subtract X days from a date using Java calendar',
 'How do I make my ArrayList Thread-Safe? Another approach to problem in Java']

In [29]:
# df= pd.read_csv("/content/drive/MyDrive/biker_data/min_5_max_10_ir_10_30k/Big_train.QApair.csv")

filtered_evaluate_corpus =[]
filtered_evaluate_answers =[]
print(len(evaluate_corpus))
print(len(evaluate_answers))
for idx,q in enumerate(evaluate_corpus):
  if not q in queries:
    filtered_evaluate_corpus.append(evaluate_corpus[idx])
    filtered_evaluate_answers.append(evaluate_answers[idx])
evaluate_corpus = filtered_evaluate_corpus
evaluate_answers = filtered_evaluate_answers
print(len(evaluate_corpus))
print(len(evaluate_answers))
#   queries_answers[idx] = str(list(eval(queries_answers[idx])))
#   df=df[~(df["title"].isin(queries))]
# evaluate_corpus=df["title"].to_list()
# evaluate_answers=df["answer"].to_list()

19532
19532
19420
19420


In [30]:
passages = evaluate_corpus
# passages = corpus
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)
# passages = evaluate_corpus
# passages = corpus

In [31]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np

# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
  tokenized_doc = []
  for token in text.lower().split():
    token = token.strip(string.punctuation)

    if len(token) > 0 and token not in stop_words.ENGLISH_STOP_WORDS:
      tokenized_doc.append(token)
  return tokenized_doc

tokenized_corpus = []
for passage in tqdm(passages):
  tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

In [32]:
#This function will search all wikipedia articles for passages that
#answer the query
def evaluate(query,answer):


  top_k=50
  answer=eval(answer)
  print(query, answer)

  bm25_scores = bm25.get_scores(bm25_tokenizer(query))
  top_n = np.argpartition(bm25_scores, -50)[-50:]
  bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
  bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

  BM25_counter = -1 
  BM25_tmp_map = 0
  BM25_tmp_mrr = 0
  temp_hits = 0

  # tmep_answer = answer[:]

  tmep_answer=[]
  for x in list(answer):
      class_api = ".".join(x.split(".")[:-1])
      tmep_answer.append(class_api)
      

  for idx, hit in enumerate(bm25_hits[0:50]):
      candidate = eval(evaluate_answers[hit['corpus_id']].replace("\n", " "))
      for c in candidate:

        class_api = ".".join(c.split(".")[:-1])

        if class_api in tmep_answer:
          if BM25_counter == -1: BM25_counter = idx + 1
          temp_hits+=1
          
          BM25_tmp_map += temp_hits/(idx+1)
          
          tmep_answer.remove(class_api)

  # print(temp_hits)
  # print(BM25_tmp_map)
  BM25_tmp_map /= len(answer)
  BM25_tmp_mrr = 0.0
  if BM25_counter!= -1:
    BM25_tmp_mrr = 1/BM25_counter



  question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
  Encoder_hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
  Encoder_hits = Encoder_hits[0]  # Get the hits for the first query

  cross_inp = [[query, passages[hit['corpus_id']]] for hit in Encoder_hits]
  cross_scores = cross_encoder.predict(cross_inp)
  for idx in range(len(cross_scores)):
      Encoder_hits[idx]['cross-score'] = cross_scores[idx]

  
  Bi_Encoder_counter = -1 
  Bi_Encoder_tmp_map = 0
  Bi_Encoder_tmp_mrr = 0

  Bi_Encoder_hit_list=[0]*top_k
  Bi_Encoder_hit_recall_list=[0]*top_k

  temp_hits = 0
  Encoder_hits = sorted(Encoder_hits, key=lambda x: x['score'], reverse=True) 
  # tmep_answer = answer[:]

  tmep_answer=[]
  for x in list(answer):
      class_api = ".".join(x.split(".")[:-1])
      tmep_answer.append(class_api)


  temp_answer1=tmep_answer[:]


  for idx,hit in enumerate(Encoder_hits[0:top_k]):
      candidate= eval(evaluate_answers[hit['corpus_id']].replace("\n", " "))
      for c in candidate:
        class_api = ".".join(c.split(".")[:-1])

        if class_api in temp_answer1:
          if not query==passages[hit['corpus_id']].replace("\n", " ").replace("?",""):
            Bi_Encoder_hit_list[idx]=1


        


        if class_api in tmep_answer:
          if not query==passages[hit['corpus_id']].replace("\n", " ").replace("?",""):


            if Bi_Encoder_counter == -1: 
              Bi_Encoder_counter = idx + 1

              
              print("\t{:.3f}\t{}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " "), evaluate_answers[hit['corpus_id']].replace("\n", " ")))
            temp_hits+=1
            
            
            Bi_Encoder_hit_recall_list[idx]=1


            Bi_Encoder_tmp_map += temp_hits/(idx+1)

            tmep_answer.remove(class_api)

  Bi_Encoder_tmp_map /= len(answer)
  Bi_Encoder_tmp_mrr = 0.0
  if Bi_Encoder_counter!= -1:
    Bi_Encoder_tmp_mrr = 1/Bi_Encoder_counter

  # answer_api= Answers[hit['corpus_id']]
  Cross_Encoder_counter = -1 
  Cross_Encoder_tmp_map = 0
  Cross_Encoder_tmp_mrr = 0

  Cross_Encoder_hit_list=[0]*top_k
  Cross_Encoder_hit_recall_list=[0]*top_k


  temp_hits = 0
  Encoder_hits = sorted(Encoder_hits, key=lambda x: x['cross-score'], reverse=True)
  # tmep_answer = answer[:]

  tmep_answer=[]
  for x in list(answer):
      class_api = ".".join(x.split(".")[:-1])
      tmep_answer.append(class_api)

  temp_answer1=tmep_answer[:]

  for idx,hit in enumerate(Encoder_hits[0:50]):
      # print("\t{:.3f}\t{}\t{}".format(hit['cross-score'], passages[hit['corpus_id']].replace("\n", " "), Answers[hit['corpus_id']].replace("\n", " ")))
      candidate= eval(evaluate_answers[hit['corpus_id']].replace("\n", " "))
      # print(candidate,answer)
      for c in candidate:
        class_api = ".".join(c.split(".")[:-1])

        if class_api in temp_answer1:
          if not query==passages[hit['corpus_id']].replace("\n", " ").replace("?",""):
            Cross_Encoder_hit_list[idx]=1


        
        if class_api in tmep_answer:
          if Cross_Encoder_counter == -1: Cross_Encoder_counter = idx + 1
          temp_hits+=1
          Cross_Encoder_hit_recall_list[idx]=1

          Cross_Encoder_tmp_map += temp_hits/(idx+1)

          tmep_answer.remove(class_api)

  Cross_Encoder_tmp_map /= len(answer)
  Cross_Encoder_tmp_mrr = 0.0
  if Cross_Encoder_counter!= -1:
    Cross_Encoder_tmp_mrr = 1/Cross_Encoder_counter

  return BM25_counter,BM25_tmp_mrr,BM25_tmp_map,Bi_Encoder_counter,Bi_Encoder_tmp_mrr,Bi_Encoder_tmp_map,Cross_Encoder_counter,Cross_Encoder_tmp_mrr,Cross_Encoder_tmp_map,str(answer),Bi_Encoder_hit_list,Bi_Encoder_hit_recall_list,Cross_Encoder_hit_list,Cross_Encoder_hit_recall_list

In [33]:
BM25_mrr = 0
BM25_map = 0

Bi_Encoder_mrr = 0
Bi_Encoder_map = 0

Cross_Encoder_mrr = 0
Cross_Encoder_map = 0

api_list =[]



Bi_Encoder_precision=[0]*4
Bi_Encoder_recall=[0]*4

Cross_Encoder_precision=[0]*4
Cross_Encoder_recall=[0]*4


for idx in range(len(queries)):

# for idx in range(5):
  
  BM25_counter,BM25_tmp_mrr,BM25_tmp_map,Bi_Encoder_counter,Bi_Encoder_tmp_mrr,Bi_Encoder_tmp_map,Cross_Encoder_counter,Cross_Encoder_tmp_mrr,Cross_Encoder_tmp_map,answer_api,Bi_Encoder_hit_list,Bi_Encoder_hit_recall_list,Cross_Encoder_hit_list,Cross_Encoder_hit_recall_list = evaluate(query = queries[idx], answer = queries_answers[idx])
  
  print(BM25_counter, Bi_Encoder_counter, Cross_Encoder_counter)

  len_api = len(eval(answer_api))
  print()
  
  temp_precision=[0]*4
  temp_recall=[0]*4
  for idx, n in enumerate([1,3,5,10]):
    # print(Bi_Encoder_hit_recall_list)
    temp_precision[idx] = sum(Bi_Encoder_hit_list[:n])/n
    temp_recall[idx] = sum(Bi_Encoder_hit_recall_list[:n])/(len_api)

  Bi_Encoder_precision = [x + y for (x, y) in zip(Bi_Encoder_precision, temp_precision)] 
  Bi_Encoder_recall = [x + y for (x, y) in zip(Bi_Encoder_recall, temp_recall)] 


  temp_precision=[0]*4
  temp_recall=[0]*4
  for idx, n in enumerate([1,3,5,10]):
    temp_precision[idx] = sum(Cross_Encoder_hit_list[:n])/n
    temp_recall[idx] = sum(Cross_Encoder_hit_recall_list[:n])/(len_api)

  Cross_Encoder_precision = [x + y for (x, y) in zip(Cross_Encoder_precision, temp_precision)] 
  Cross_Encoder_recall = [x + y for (x, y) in zip(Cross_Encoder_recall, temp_recall)] 


  # print(BM25_tmp_mrr, Bi_Encoder_tmp_mrr, Cross_Encoder_tmp_mrr)
  # print(BM25_tmp_map, Bi_Encoder_tmp_map, Cross_Encoder_tmp_map)

  api_list.append(answer_api)
  BM25_mrr+=BM25_tmp_mrr
  BM25_map+=BM25_tmp_map

  Bi_Encoder_mrr+=Bi_Encoder_tmp_mrr
  Bi_Encoder_map+=Bi_Encoder_tmp_map
  
  Cross_Encoder_mrr+=Cross_Encoder_tmp_mrr
  Cross_Encoder_map+=Cross_Encoder_tmp_map

BM25_mrr/=len(queries)
BM25_map/=len(queries)

Bi_Encoder_mrr/=len(queries)
Bi_Encoder_map/=len(queries)

Cross_Encoder_mrr/=len(queries)
Cross_Encoder_map/=len(queries)

Bi_Encoder_precision = [x/len(queries) for x in Bi_Encoder_precision]
Bi_Encoder_recall = [x/len(queries) for x in Bi_Encoder_recall]


Cross_Encoder_precision = [x/len(queries) for x in Cross_Encoder_precision]
Cross_Encoder_recall = [x/len(queries) for x in Cross_Encoder_recall]

print("Bi_Encoder_precision")
print(Bi_Encoder_precision)

print("Bi_Encoder_recall")
print(Bi_Encoder_recall)

print("Cross_Encoder_precision")
print(Cross_Encoder_precision)

print("Cross_Encoder_recall")
print(Cross_Encoder_recall)

print(BM25_mrr,Bi_Encoder_mrr,Cross_Encoder_mrr)
print(BM25_map,Bi_Encoder_map,Cross_Encoder_map)
print(len(list(set(api_list))))

How to convert binary string value to decimal ['java.lang.Integer.parseInt']
	0.789	Converting String binary to integer	['java.lang.Integer.valueOf', 'java.lang.Integer.parseInt']
2 1 1

How can I use html styles in strings with format arguments ['java.lang.String.format']
	0.724	How to format a string with a variable number of arguments	['java.lang.String.join']
1 2 1

Converting a sentence string to a string array of words in Java ['java.lang.String.split']
	0.784	Convert Java String to Array	['java.lang.String.split']
1 2 4

How do I create a parent-last / child-first ClassLoader in Java, or How to override an old Xerces version that was already loaded in the parent CL ['java.lang.Class.forName']
	0.626	How do you include a class into the runtime environment when a custom classloader loaded the class?	['java.lang.Class.newInstance']
1 2 2

How to modify a Collection while iterating using for-each loop without ConcurrentModificationException ['java.util.Iterator.remove']
	0.661	How t

In [35]:

print("Bi_Encoder_precision")
for x in Bi_Encoder_precision:
  print(x) 

print("Bi_Encoder_recall")
for x in Bi_Encoder_recall:
  print(x)

print("Cross_Encoder_precision")
for x in Cross_Encoder_precision:
  print(x)

print("Cross_Encoder_recall")
for x in Cross_Encoder_recall:
  print(x)

Bi_Encoder_precision
0.36904761904761907
0.44047619047619035
0.426984126984127
0.40952380952380935
Bi_Encoder_recall
0.3611111111111111
0.7202380952380952
0.8214285714285714
0.9226190476190477
Cross_Encoder_precision
0.5476190476190477
0.6190476190476194
0.6023809523809521
0.571428571428571
Cross_Encoder_recall
0.7003968253968254
0.878968253968254
0.9265873015873016
0.9543650793650794


In [ ]:
BM25_counter,BM25_tmp_mrr,BM25_tmp_map,Bi_Encoder_counter,Bi_Encoder_tmp_mrr,Bi_Encoder_tmp_map,Cross_Encoder_counter,Cross_Encoder_tmp_mrr,Cross_Encoder_tmp_map,answer_api = search(query = "Java Convert File to Byte Array and vice versa", answer = "['java.nio.file.Files.write', 'java.nio.file.Files.readAllBytes']")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#This function will search all wikipedia articles for passages that
#answer the query
def get_cross_encoder_result(query,answer):

  ##### Sematic Search #####
  #Encode the query using the bi-encoder and find potentially relevant passages
  question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
  # question_embedding = question_embedding.cuda()
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
  hits = hits[0]  # Get the hits for the first query

  #Output of top-5 hitt
  # print("Top-5 Bi-Encoder Retrieval hits")
  hits = sorted(hits, key=lambda x: x['score'], reverse=True)
  # print(len(hits))
  result_list = []
  added=False
  for idx,hit in enumerate(hits[0:50]):
      passage = passages[hit['corpus_id']]
      if evaluate_answers[hit['corpus_id']].replace("\n", " ") == answer:
        if added == False:
          result_list.append([query,passage,1])
        added=True
  counter=0
  for idx,hit in enumerate(hits[0:50]):
      passage = passages[hit['corpus_id']]
      if evaluate_answers[hit['corpus_id']].replace("\n", " ") != answer:
        if counter<10:
          result_list.append([query,passage,0])
        counter+=1

  return result_list
evaluate_dataset =[]
for idx in range(len(queries)):
  
  result_list = get_cross_encoder_result(query = queries[idx], answer = queries_answers[idx])
  evaluate_dataset+=result_list
df_cross_encoder = pd.DataFrame(data=evaluate_dataset,columns=["query","passage","is_relevent"])
df_cross_encoder.to_csv("/content/drive/MyDrive/biker_data/min_5_max_10_ir_10_30k/cross_encoder_eval.csv")